In [128]:
#Checkers Graph
import networkx as nx
from pyvis.network import Network
from functools import reduce

#0b010 = black
#0b001 = white
#0b100 | player = king (IE 0b101 = black king)

class CheckersGame:
    def __init__(self):
        self.graph = nx.DiGraph()
        for r in range(0,8):
            o = (r+1)%2
            player = 2 if r <3 else 1 if r > 4 else 0
            for c in range(0,8)[o:8:2]:
                self.graph.add_node(f"{chr(ord('`')+r+1)}{c}",player = player, loc=(r,c), group=player)
            
        for r in range(0,7):
            o = (r+1)%2
            for c in range(0,8)[o:8:2]:
                _from = f"{chr(ord('`')+r+1)}{c}"
                _to1 = f"{chr(ord('`')+r+2)}{c+1}"
                _to2 = f"{chr(ord('`')+r+2)}{c-1}"
                if(c+1 < 8):
                    self.graph.add_edge(_from,_to1,mask=0b010)
                if(c-1 >= 0):
                    self.graph.add_edge(_from,_to2,mask=0b010)
        
        for r in range(0,8)[8:0:-1]:
            o = (r+1)%2
            for c in range(0,8)[o:8:2]:
                _from = f"{chr(ord('`')+r+1)}{c}"
                _to1 = f"{chr(ord('`')+r)}{c+1}"
                _to2 = f"{chr(ord('`')+r)}{c-1}"
                if(c+1 < 8):
                    self.graph.add_edge(_from,_to1,mask=0b001)
                if(c-1 >= 0):
                    self.graph.add_edge(_from,_to2,mask=0b001)



game = CheckersGame()
g = game.graph

# nt = Network(notebook=True, cdn_resources="remote")
# nt.from_nx(game.graph)
# nt.show('test.html')
# print(game.graph.edges)
# print(game.graph.nodes)
p1_nodes = [n for n,d in g.nodes(data=True) if d['player'] == 2]
# print(p1_nodes)
# print(game.graph.get_edge_data('a1','b2'))

validMoves = []
validJumps = []

for node in p1_nodes:
    p = g.nodes(data=True)[node]['player']
    edges = [f'{node}->{e[1]}' for e in g.edges(node,data=True) if e[2]['mask'] & p == p and g.nodes(data=True)[e[1]]['player'] == 0]
    if(len(edges)>0):
        validMoves.extend(edges)

g.nodes(data=True)['d2']['player'] = 1
g.nodes(data=True)['d2']['group'] = 1

g.nodes(data=True)['f2']['player'] = 0
g.nodes(data=True)['f2']['group'] = 0

#check for jumps
for node in p1_nodes:
    p = g.nodes(data=True)[node]['player']
    for n in [n for n in g[node] if g.nodes(data=True)[n]['player'] == 1 and g.get_edge_data(node,n)['mask']&p!=0]:
        print(node,n)
        print(g.nodes(data=True)[node])
        print(g.nodes(data=True)[n])
        print([e for e in g.edges(n,data=True) if e[2]['mask']&p==p])

# print(validMoves)
# print(validJumps)
# center = 'c1'
# print(list(iter(g[center])))
# local_group = [center, *g.neighbors(center), *reduce(lambda a,b: a+b,([list(g.neighbors(a)) for a in g.neighbors(center)]))]
# sub = g.subgraph(local_group)


# print(reduce(lambda a,b: a+b,([list(g.neighbors(a)) for a in g.neighbors('b2')])))

# nt = Network(notebook=True, cdn_resources="remote")
# nt.from_nx(g)
# nt.show('test.html')

#subgraph
# player x -> mask&player x -> player=0



#a1 -> b0
#a1 -> b2




c1 d2
{'player': 2, 'loc': (2, 1), 'group': 2}
{'player': 1, 'loc': (3, 2), 'group': 1}
[('d2', 'e3', {'mask': 2}), ('d2', 'e1', {'mask': 2})]
c3 d2
{'player': 2, 'loc': (2, 3), 'group': 2}
{'player': 1, 'loc': (3, 2), 'group': 1}
[('d2', 'e3', {'mask': 2}), ('d2', 'e1', {'mask': 2})]


In [137]:
#game
from collections import namedtuple
from io import StringIO
from typing import NamedTuple
from attr import dataclass
import numpy as np


class checkers:
    def __init__(self, height=8, width =8):
        self.board = np.zeros((8,8),dtype=np.int8)
        self.height = height
        self.width = width
        self.board[0:3:2, 1:8:2] = 1
        self.board[1, 0:8:2] = 1
        self.board[6:8:2, 1:8:2] = 2
        self.board[5:8:2, 0:8:2] = 2
        self.winner = 0

    @dataclass
    class Movement:
        start:NamedTuple("start",[('row',int), ('col',int)])
        end:NamedTuple("end",[('row',int), ('col',int)])
        jump:NamedTuple("jump",[('row',int), ('col',int)]) = None
        subsequent: list = None
        
        def __toChar(self,position):
            if position == None: return ""
            retval = chr(ord('`')+position[0]+1), str(position[1])
            return retval

        def __str__(self):
            val = ''.join(self.__toChar(self.start)) +'->'+''.join(self.__toChar(self.end))+('∩'+''.join(self.__toChar(self.jump)) if self.jump else '')
            return val
        
        def __repr__(self):
            return self.__str__()
        
    def _posToStr(self,position):
        if position == None: return ""
        return chr(ord('`')+position[0]+1) + str(position[1])
    
    def _strToPos(self,position):
        if position == None: return ""
        return int(ord(position[0])-ord('a')), int(position[1])

    def __valid(self,position):
        if position == None: return False
        r,c = position
        return r >= 0 and r < self.height and c >= 0 and c < self.width

    def _jumps(self,position, player = None, allowBackJump = True, exclude_from = (-1,-1)):
        if type(position) == type('str'): position = self._strToPos(position)
        jumps = []
    
        r,c = position
        if (player:= (player or self.board[position])) == 0:  return []
        opponent = 3^(player&3)
        if player&4 == 4: allowBackJump = True
        directions = []
        if player   & 4 == 4:  directions =  [(1,1),(2,2)], [(1,-1),(2,-2)], [(-1,1),(-2,2)], [(-1,-1),(-2,-2)]
        elif player & 1 == 1:  directions =  [(1,1),(2,2)], [(1,-1),(2,-2)]
        elif player & 2 == 2:  directions =  [(-1,1),(-2,2)], [(-1,-1),(-2,-2)] 
        else: return jumps
        
        for neighbor,drop in directions:
            n = (r+neighbor[0],c+neighbor[1])
            d = (r+drop[0],c+drop[1])
            if n == exclude_from or d==exclude_from: continue
            if(not self.__valid(n) or not self.__valid(d)): continue

            if (self.board[n]&3)==opponent and self.board[d] == 0:
                jump_move = self.Movement(position,d,n) 
                jumps.append(jump_move)
                p = (player&3|4) if allowBackJump else player
                multi_jumps = self._jumps(d, player = p, allowBackJump=allowBackJump, exclude_from = position)
                jump_move.subsequent = multi_jumps
        return jumps
         
    def _moves(self,position) -> list:
        if type(position) == type('str'): position = self._strToPos(position)
        row, col = position
        if (player:= self.board[position]) == 0:  return []
        directions = []
        if   player&4==4: directions = [(1,-1),(1,1),(-1,-1),(-1,1)] 
        elif player&2==2: directions = [(-1,-1),(-1,1)]
        elif player&1==1: directions = [(1,-1),(1,1)]

        for dr,dc in directions:
           if self.__valid((row+dr,col+dc)) and self.board[row+dr,col+dc] == 0: yield self.Movement(position,(row+dr,col+dc))
        
    def move(self,move:Movement):
        x1,y1 = move.start
        x2,y2 = move.end
        jx,jy = -1,-1 or move.jump
        if(move.jump): 
            self.jump(move)
            p = self.board[x2,y2]
        else:
            p = self.board[x1,y1]
            self.board[x1,y1] = 0
            self.board[x2,y2] = p

        if p&3 == 1 and x2 == self.height-1: self.board[x2,y2] = 5
        if p&3 == 2 and x2 == 0: self.board[x2,y2] = 6
        return move
    
    def jump(self,move:Movement):
        x1,y1 = move.start
        x2,y2 = move.end
        jx,jy = move.jump
        p = self.board[x1,y1]
        self.board[x1,y1] = 0
        self.board[x2,y2] = p
        self.board[jx,jy] = 0
        return move

    def __str__(self):
        # https://stackoverflow.com/questions/4842424/list-of-ansi-color-escape-sequences
        bold =  '\033[1m'
        borderColor = bold+'\033[90m'
        brightBorderColor = bold+'\033[37m'
        end_color = '\033[0m'
        underline = '\033[4m'
        red = bold+'\033[91m'
        green = bold+'\033[92m'
        out = StringIO()
        print('    ' + underline,end="",file=out)
        for i in range(self.board.shape[1]):
            print(f"   {i}  ",end="",file=out)
        print(' '+ end_color,file=out)
        for row in range(self.board.shape[0]):
            print("|",end="",file=out)
            print(" "*3,end="",file=out)
            print(f"{borderColor}+-----"*8, end=f"+\n",file=out)
            print(end_color + f"|{  chr(ord('`')+row+1)  }  "+ borderColor,end="",file=out)
            for col in range(self.board.shape[1]):
                color = red if self.board[row,col]&3&1 == 1 else green if self.board[row,col]&3&2 == 2 else borderColor
                char = 'K' if self.board[row,col] & 4 == 4 else '@' if self.board[row,col] != 0 else ' '
                print(f"| {color} {char}  {borderColor}",end="",file=out)
            print(f"|{end_color}",file=out)
                
        print("|",end="",file=out)
        print(" "*3,end="",file=out)
        print(f"{borderColor}+-----"*8, end=f"+{end_color}\n",file=out)
        return out.getvalue()
    
    def __repr__(self) -> str:
        return self.__str__()  


In [136]:
5&3&1

1

In [94]:
#move tree
from treelib import Tree
from treelib import Node as nd

def treeFrom(position, movements:list[checkers.Movement]):
    def r_build_tree(t:Tree, n:nd):
        if n.data == None: return
        for e in n.data:
            _n = t.create_node(str(e),parent=n, data = e.subsequent)
            r_build_tree(t,_n)
            
    if movements == None or len(movements) == 0: return None

    t=Tree() 
    _node = t.create_node(position,data = movements)
    r_build_tree(t,_node)
    return t

In [101]:
game = checkers()
game.move(checkers.Movement((2,1),(3,4)))
game.move(checkers.Movement((5,4),(4,3)))
game.move(checkers.Movement((5,2),(4,1)))
game.move(checkers.Movement((7,0),(3,2)))

mv = list(game._moves('d2'))
mv.extend(list(game._jumps('d2',allowBackJump=False)))

t = treeFrom('d2',mv)
if(t): t.show()


print(game)

d2
└── d2->c1

       0     1     2     3     4     5     6     7   
|   +-----+-----+-----+-----+-----+-----+-----+-----+
|a  |     |  @  |     |  @  |     |  @  |     |  @  |
|   +-----+-----+-----+-----+-----+-----+-----+-----+
|b  |  @  |     |  @  |     |  @  |     |  @  |     |
|   +-----+-----+-----+-----+-----+-----+-----+-----+
|c  |     |     |     |  @  |     |  @  |     |  @  |
|   +-----+-----+-----+-----+-----+-----+-----+-----+
|d  |     |     |  @  |     |  @  |     |     |     |
|   +-----+-----+-----+-----+-----+-----+-----+-----+
|e  |     |  @  |     |  @  |     |     |     |     |
|   +-----+-----+-----+-----+-----+-----+-----+-----+
|f  |  @  |     |     |     |     |     |  @  |     |
|   +-----+-----+-----+-----+-----+-----+-----+-----+
|g  |     |  @  |     |  @  |     |  @  |     |  @  |
|   +-----+-----+-----+-----+-----+-----+-----+-----+
|h  |     |     |  @  |     |  @  |     |  @  |     |
|   +-----+-----+-----+-----+-----+-----+-----+-----+



In [148]:
#simulate Game
#just take random moves for each player
import random
import IPython.display as display
game = checkers()



while True:
    redMoves = []
    redPlayers = list(zip(*np.where(game.board&3&1 == 1)))
    for p in redPlayers:
        redMoves.extend(list(game._moves(p)))
        redMoves.extend(list(game._jumps(p,allowBackJump=False)))
    if(len(redMoves) == 0): break
    i = input()
    if i == 'q': break
    display.clear_output(False)
    print('red')
    rm = random.choice(redMoves)
    print(list(map(str,redMoves)))
    game.move(rm)
    print(rm)
    while rm.subsequent != None and len(rm.subsequent) > 0:
        rm = random.choice(rm.subsequent)
        game.move(rm)
   
    print(game)

    greenMoves = []
    greenJumps = []

    greenPlayers = list(zip(*np.where(game.board&3&2 == 2)))
    for p in greenPlayers:
        greenMoves = (list(game._moves(p)))
        greenJumps = (list(game._jumps(p,allowBackJump=False)))

    if(len(greenMoves) == 0 and len(greenJumps)==0): 
        "breaking"
        break

    i = input()
    if i == 'q': break
    display.clear_output(False)
    print('green')
    print(list(map(str,greenMoves)))
    
    if len(greenJumps) > 0:
        gm = random.choice(greenJumps)
    else:
        gm = random.choice(greenMoves)
 
    game.move(gm)
    print(gm)
    while gm.subsequent != None and len(gm.subsequent) > 0:
        gm = random.choice(gm.subsequent)
        game.move(gm)
        print(gm)
    
    
    print(game)



red
['c1->d0', 'c1->d2', 'c3->d2', 'c3->d4', 'c5->d4', 'c5->d6', 'c7->d6']
c1->d0
       0     1     2     3     4     5     6     7   
|   +-----+-----+-----+-----+-----+-----+-----+-----+
|a  |     |  @  |     |  @  |     |  @  |     |  @  |
|   +-----+-----+-----+-----+-----+-----+-----+-----+
|b  |  @  |     |  @  |     |  @  |     |  @  |     |
|   +-----+-----+-----+-----+-----+-----+-----+-----+
|c  |     |     |     |  @  |     |  @  |     |  @  |
|   +-----+-----+-----+-----+-----+-----+-----+-----+
|d  |  @  |     |     |     |     |     |     |     |
|   +-----+-----+-----+-----+-----+-----+-----+-----+
|e  |     |     |     |     |     |     |     |     |
|   +-----+-----+-----+-----+-----+-----+-----+-----+
|f  |  @  |     |  @  |     |  @  |     |  @  |     |
|   +-----+-----+-----+-----+-----+-----+-----+-----+
|g  |     |  @  |     |  @  |     |  @  |     |  @  |
|   +-----+-----+-----+-----+-----+-----+-----+-----+
|h  |  @  |     |  @  |     |  @  |     |  @  |     |
